In [1]:
from pyrocko import io,model
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# data folder (response removed)
workdir='../'
datadir =  os.path.join(workdir,'DATA_VLP_response')

# events catalogue
catdir=os.path.join(workdir,'CAT')
catname=os.path.join(catdir,'catalogue_flegrei_VLP_gilberto.pf')
target_events=model.load_events(catname)

# stations list
stationsdir=os.path.join(workdir,'META_DATA')
stationsname=os.path.join(stationsdir,'stations_flegrei_INGV_final_near.pf')
stations=model.load_stations(stationsname)

# new file dir
newcatdir=os.path.join(catdir,'CRACK')

for targ_ev in target_events:
    event_name=targ_ev.name
    ev_dir = os.path.join(datadir, event_name)
    
    if not os.path.isdir(ev_dir):
        print(event_name, 'missing report dir', ev_dir)
    elif os.path.isdir(ev_dir):
        file_name = os.path.join(ev_dir,event_name + '.mseed' )
        traces = io.load(file_name)
        print(f'reading event: {event_name}')
        stations_dict = {key.station: [] for key in stations}
        max_values = {st.station: [] for st in stations}
        # add lat lon elevation
        for st in stations:
            max_values[st.station].append(st.lat)
            max_values[st.station].append(st.lon)
            max_values[st.station].append(st.elevation)

        for tr in traces:
            #filter trace
            tmp_filt_trace= tr
            tmp_filt_trace.lowpass(4,0.124)
            tmp_filt_trace.highpass(4,0.104)
            try:
                stations_dict[tr.station].append(tmp_filt_trace)
            except:
                continue
        for key in stations_dict:
            if len(stations_dict[key])==3:          # chack if stations has all 3 components
                # calculate vector length (r)
                e = stations_dict[key][0].ydata
                n = stations_dict[key][1].ydata
                z = stations_dict[key][2].ydata
                if len(e)==len(n)==len(z):
                    r = np.sqrt( e**2 + n**2 + z**2 )
                else:
                    min_len = np.min( [len(e),len(n),len(z)] )
                    r = np.sqrt( e[0:min_len]**2 + n[0:min_len]**2 + z[0:min_len]**2 )
                    print(f'WARNING: DIFFERENT TRACES LENGTH\nlen E : {len(e)}, len N : {len(n)}, len Z : {len(z)}')
                # find r MAX
                tmp_max=np.max(r)
                tmp_min=np.min(r)
                if tmp_max>np.abs(tmp_min):
                    rmax=tmp_max
                else:
                    rmax=tmp_min
                tmax=np.where(r==rmax)[0][0]
                emax=e[tmax]
                nmax=n[tmax]
                zmax=z[tmax]
                max_values[key].append(emax)
                max_values[key].append(nmax)
                max_values[key].append(zmax)

        # save file
        
        new_filename=os.path.join(newcatdir,event_name +'_max_displacement.txt')
        events_txt=[]
        for key in max_values:
            if len(max_values[key])==6:     # station has recorded traces    
                name=key
                lat= str(max_values[key][0])
                lon=str(max_values[key][1])
                elev=str(max_values[key][2])
                d_east=str(max_values[key][3])
                d_nord=str(max_values[key][4])
                d_z=str(max_values[key][5])
                events_txt.append(f'{name},{lat},{lon},{elev},{d_east},{d_nord},{d_z}')

        with open(new_filename, 'w') as output:
            output.write('station_name,latitude,longitude,elevation[m],d_east[m],d_nord[m],d_z[m]\n')
            for row in events_txt:
                output.write(str(row) + '\n')

reading event: flegrei_2023_06_11_06_44_25
reading event: flegrei_2023_09_07_17_45_28
reading event: flegrei_2023_09_26_07_10_29
reading event: flegrei_2023_10_02_20_08_26
reading event: flegrei_2024_04_27_03_44_56
reading event: flegrei_2024_05_22_06_28_00
reading event: flegrei_2024_06_08_01_52_04
reading event: flegrei_2024_06_18_01_58_24
reading event: flegrei_2024_07_26_11_46_21
reading event: flegrei_2024_08_30_19_23_15
len E : 60000, len N : 60001, len Z : 60001


In [ ]:
for key in max_values:
    print(len(max_values[key]))